In [1]:
import os 
import glob
from pathlib import Path
from rtmag.paper.parse import parse_tai_string
from sunpy.map import Map
import numpy as np
from skimage.transform import resize
from rtmag.paper.load import load_input_label, MyModel

In [2]:
data_path = "/mnt/f/sdo_AR/NOAA11158_HARP377/hmi"
hmi_p_files = sorted(glob.glob(os.path.join(data_path, '*Bp.fits')))  # x
hmi_t_files = sorted(glob.glob(os.path.join(data_path, '*Bt.fits')))  # y
hmi_r_files = sorted(glob.glob(os.path.join(data_path, '*Br.fits')))  # z
data_paths = list(zip(hmi_p_files, hmi_t_files, hmi_r_files))

In [3]:
ix = 240
data_path = data_paths[ix]
data_path

('/mnt/f/sdo_AR/NOAA11158_HARP377/hmi/hmi.sharp_cea_720s.377.20110214_000000_TAI.Bp.fits',
 '/mnt/f/sdo_AR/NOAA11158_HARP377/hmi/hmi.sharp_cea_720s.377.20110214_000000_TAI.Bt.fits',
 '/mnt/f/sdo_AR/NOAA11158_HARP377/hmi/hmi.sharp_cea_720s.377.20110214_000000_TAI.Br.fits')

In [4]:
input_files, label_files = load_input_label("/mnt/f/isee_dataset/11158")
len(input_files), len(label_files)

(66, 66)

In [5]:
idx = 13
input_files[idx], label_files[idx]

(PosixPath('/mnt/f/isee_dataset/11158/input/input_11158_20110214_000000.npz'),
 PosixPath('/mnt/f/isee_dataset/11158/label/label_11158_20110214_000000.npz'))

In [6]:
from time import time

In [7]:
mm = MyModel("/home/mgjeon/space/workspace/base_important/uno_pi_cc_hnorm_unit_aug/model_75.pt")

Model loaded from epoch 75


In [8]:
# tstr = Path(data_path[0]).name[23:38]
# obstime = parse_tai_string(tstr)
hmi_p, hmi_t, hmi_r, = data_path
p_map, t_map, r_map = Map(hmi_p), Map(hmi_t), Map(hmi_r)
maps = [p_map, t_map, r_map]
hmi_data = np.stack([maps[0].data, -maps[1].data, maps[2].data]).transpose()
hmi_data = np.nan_to_num(hmi_data, nan=0.0)
hmi_data = hmi_data.astype(np.float32)

# ox, oy, _ = hmi_data.shape
nx, ny = 512, 256

# l = 0.36 # Mm

# dx = (ox * l)/nx
# dy = (oy * l)/ny
# dz = dy 

# dx, dy, dz = dx * 1e8, dy * 1e8, dz * 1e8  # cm
# dV = dx * dy * dz # cm^3

model_input = resize(hmi_data, (nx, ny, 3))
model_input = model_input[None, :, :, None, :]
model_input = model_input.transpose(0, 3, 2, 1, 4)
model_input.shape

See https://docs.sunpy.org/en/stable/code_ref/map.html#fixing-map-metadata for how to fix metadata before loading it with sunpy.map.Map.
See https://fits.gsfc.nasa.gov/fits_standard.html for the FITS unit standards. [sunpy.map.mapbase]


(1, 1, 256, 512, 3)

In [9]:
b = mm.get_pred_from_numpy(model_input)

Model loaded from epoch 75


In [10]:
B = mm.get_label(label_files[idx])

In [11]:
import rtmag.paper.metric as m

In [13]:
start_time = time()
res = m.evaluate(b, B)
print(f"{time() - start_time} seconds")

C_vec     : 0.96
C_cs      : 0.95
E_n'      : 0.68
E_m'      : 0.58
eps       : 0.94
1.235198736190796 seconds
